<a href="https://colab.research.google.com/github/RautRitesh/langgraph/blob/main/Lanngraph_corrective_rag_self_rag_and_agentic_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## In this notebook we will try to create the production ready different RAG approaches that langgraph supports

### We will start with creating a corrective rag which will grade the document and generate the best answer

In [132]:
pip install tavily-python langchain-tavily


In [38]:
pip install langchain langgraph langchain-core langchain-community langchain-groq langchain faiss-cpu langchain-huggingface

In [133]:
# let us understand how the document grader works first by creating the retriever using faiss

from langchain_community.document_loaders import TextLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_tavily import TavilySearch


In [65]:
loader=TextLoader("/content/nepal_insurance_details.txt")
docs=loader.load()

In [66]:
embeddings=HuggingFaceEmbeddings(model="sentence-transformers/all-MiniLM-L6-v2")

In [67]:
text_splitter=RecursiveCharacterTextSplitter(chunk_overlap=200, chunk_size=800,separators=["\n\n","\n"])
chunk=text_splitter.split_documents(docs)
vector_stores=FAISS.from_documents(chunk, embeddings)

In [68]:
retriever=vector_stores.as_retriever(search_kwargs={"k":4})

In [69]:
result=retriever.invoke("What is the document about?")

In [70]:
print(result)

[Document(id='9b9faabb-8c82-4cdc-b981-a9dfad993254', metadata={'source': '/content/nepal_insurance_details.txt'}, page_content='Enrollment Procedure\nCitizens enroll via local Enrollment Assistants (volunteers) at ward offices, health posts, or empaneled facilities using the Insurance Management Information System (IMIS). Provide citizenship certificate, photos, and pay NPR 3,500 annually for a standard family (recent figure); receive a health insurance card instantly or shortly after.'), Document(id='0ee19dc2-db7f-46f7-b160-bcb243f40aa0', metadata={'source': '/content/nepal_insurance_details.txt'}, page_content='Description of the tests provided free of charge by specific government programs (outside of the standard Health Insurance Board coverage) are:'), Document(id='6d142804-d67e-424b-bb23-6aa3c8f22162', metadata={'source': '/content/nepal_insurance_details.txt'}, page_content='Pay Co-Pay: Cover 20% of approved costs immediately (e.g., NPR 200 on a NPR 1,000 bill); no upfront payme

In [71]:
for doc in result:
  print(doc.id)
  print(doc.metadata)
  print(doc.page_content)
  print("\n\n-------")

9b9faabb-8c82-4cdc-b981-a9dfad993254
{'source': '/content/nepal_insurance_details.txt'}
Enrollment Procedure
Citizens enroll via local Enrollment Assistants (volunteers) at ward offices, health posts, or empaneled facilities using the Insurance Management Information System (IMIS). Provide citizenship certificate, photos, and pay NPR 3,500 annually for a standard family (recent figure); receive a health insurance card instantly or shortly after.


-------
0ee19dc2-db7f-46f7-b160-bcb243f40aa0
{'source': '/content/nepal_insurance_details.txt'}
Description of the tests provided free of charge by specific government programs (outside of the standard Health Insurance Board coverage) are:


-------
6d142804-d67e-424b-bb23-6aa3c8f22162
{'source': '/content/nepal_insurance_details.txt'}
Pay Co-Pay: Cover 20% of approved costs immediately (e.g., NPR 200 on a NPR 1,000 bill); no upfront payment for fully covered vulnerable groups like seniors over 70.
Get Treatment: Receive services; hospital bi

## This is the data that we get from the retriever, now we will use this data to check if it is correct or not than only we will go with the generation ,hence this is the implementation of the corrective rag

# Using and settin up the data now

In [117]:
from langchain_groq import ChatGroq
from langgraph.graph import MessagesState, StateGraph , START , END
from langgraph.prebuilt import ToolNode
from typing import Literal
from langchain_core.tools import tool
from pydantic import BaseModel , Field
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [131]:
from google.colab import userdata
api_key=userdata.get('groq_api_key')
tavily_api_key=userdata.get('Tavily_api_key')

In [74]:
llm=ChatGroq(model="openai/gpt-oss-20b",api_key=api_key)

In [75]:
class AgentState(MessagesState):
  document:list[str]

In [103]:
class GradeDocument(BaseModel):
  binary_score:Literal["yes","no"]=Field(description="Choose yes or no based the user question  related to medical or not")

In [118]:
llm_with_structured_output=llm.with_structured_output(GradeDocument)

In [113]:
system="""You are an assistant used for grading the document and the user question are related with each other or not , if related than
set it as 'yes' else 'no' based on the document provided by the user"""
prompt=ChatPromptTemplate([
    ("system",system),
    ("human","Based on this document{document} for this question {question}")
])
retreiver_grader=prompt | llm_with_structured_output

In [125]:
system_2="""You are an advance query rewriter that transform the user query to the best query for tavily search first understand the user query and generate the best alternative, rewrite the question closer to the acutal one"""
prompt_query_rewriter=ChatPromptTemplate([
    ("system",system_2),
    ("human","Generate the best question for this {question}")
])
query_rewriter=prompt_query_rewriter|llm|StrOutputParser()


In [81]:
def retriever_rag(state:AgentState):
  query=state["messages"][-1]
  docs=retriever.invoke(query)
  return {"document":docs}


In [128]:
def router1(state:AgentState)->Literal["generator","query_rewriter"]:
  user_query=state["messages"][-1]
  docs=state["document"]
  filtered_docs=[]
  for doc in docs:
    filtered_docs.append(doc.page_content)
  response=retreiver_grader.invoke({"question":{user_query},"document":{filtered_docs}})
  if response.binary_score== "yes":
    return "generator"
  else:
    return "query_rewriter"


In [129]:
def query_rewriter(state:AgentState):
  user_query=state["messages"][-1]
  new_query=query_rewriter.invoke({"question":{user_query}})
  return {"messages":[new_query]}

In [134]:
tavily_search=TavilySearch(tavily_api_key=tavily_api_key)

In [137]:
@tool
def search(query:str):
  """This is a tool used for doing search on the internet"""
  response=tavily_search.invoke(query)
  return response
